# Imports

In [39]:
#####################################################
################## PACKAGES #########################
#####################################################
import pandas as pd
import sys 
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
from sklearn.metrics import accuracy_score, roc_auc_score
from datetime import datetime, timedelta, date
from itertools import combinations
from numpy.linalg import norm
import pickle
import os
import torch.nn as nn
import torch
from torch.nn import Linear
import torch.nn.functional as F
import altair as alt
from altair import expr, datum
from vega_datasets import data
from geopy.geocoders import Nominatim
from tqdm import tqdm

tqdm.pandas()
DATA_PATH = './data'

# First infos

In [43]:
# metadata cleaning
def preprocess_metadata(read=True, DATA_PATH=DATA_PATH):
    """process the metadata

    Args:
        metadata (pandas dataframe): the metadata
    """
    if read:
        metadata = pd.read_pickle(f'{DATA_PATH}/metadata_processed.pkl')
    else:
        pd.read_csv(f'{DATA_PATH}/metadata.csv')
        metadata['date'] = pd.to_datetime(metadata['date'], format='%Y%m%d')
        metadata = metadata.drop(columns=['set'])

        # Create an instance of the geocoder
        geolocator = Nominatim(user_agent="my-app")

        # Use reverse geocoding to get the location information
        metadata['location'] = metadata.progress_apply(lambda x: geolocator.reverse((x.lat, x.lon)), axis=1)

        # Extract the country from the location information
        metadata['country'] = metadata.location.progress_apply(lambda x: x.raw['address'].get('country'))
        metadata['city'] = metadata.location.progress_apply(lambda x: x.raw['address'].get('city'))
        
        #save as pkl 
        metadata.to_pickle(f'{DATA_PATH}/metadata_processed.pkl')

    return metadata

metadata = preprocess_metadata()

In [26]:
## The geographic distribution
countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project(
    "equirectangular"
).properties(
    width=500,
    height=300
)

points = alt.Chart(metadata).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.value(10),
    tooltip='date'
)

background+points

alt.LayerChart(...)

In [67]:
# COUNTRY DISTRIBUTION
country_distrib = metadata.groupby('country').agg(count=('date', 'count')).reset_index()
country_distrib = country_distrib.sort_values(by='count', ascending=False)
hist_country = alt.Chart(country_distrib.sort_values(by='count')).mark_bar().encode(
    x='country:N',
    y=alt.Y('count:Q')
)
hist_country.properties(title='Distribution of the data')

alt.Chart(...)

In [72]:
#proportion of true and false
metadata.plume.value_counts()

yes    216
no     214
Name: plume, dtype: int64